In [28]:
import cloudscraper

In [29]:
from bs4 import BeautifulSoup
import geopandas as gpd
import pandas as pd

## Open State data

In [30]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_gdf = gpd.read_file(file_path)

## Open Population Data

In [31]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [32]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [33]:
states_with_population_gdf = states_gdf.merge(
    state_populations_df, on="NAME", how="left"
)
states_with_population_gdf = states_with_population_gdf[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get Vehicle Data

In [79]:
scraper = cloudscraper.create_scraper()
r = scraper.get("https://ammo.com/articles/gun-ownership-by-state")

In [80]:
soup = BeautifulSoup(r.text, "html.parser")

In [81]:
trs = soup.find("table").find_all("tr")[1:]

In [82]:
data = []
for tr in trs[1:]:
    tds = tr.find_all("td")
    name = tds[0].text[3:].strip()
    rate = float(tds[1].text)
    data.append({"NAME": name, "RATE": rate})

In [83]:
df = pd.DataFrame(data)

## Merge Data

In [84]:
gun_gdf = states_with_population_gdf.merge(df, on="NAME", how="inner")

In [85]:
gun_gdf = gun_gdf.to_crs(9311)
gun_gdf.to_file("data/Gun_Ownership_Per_State.gpkg")